In [1]:
import sqlite3
from tkinter import *
from tkinter.ttk import Combobox
from tkinter import messagebox
from datetime import datetime
import time
import nltk
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob.classifiers import NaiveBayesClassifier

In [13]:
df=pd.read_json("d:/ducat/mlfiles/news.json",lines=True)
df=df[['category','headline']]
df=df[df.category.isin(['POLITICS','ENTERTAINMENT','COMEDY'])]
corpus=df.headline.values
target=df.category.values
sw=stopwords.words('english')
sw.remove('not')
sw.remove("hadn't")
sw.remove("wasn't")
sw.remove("don't")
sw.remove("doesn't")
def text_cleaning(doc):
    doc=doc.lower()
    doc=re.sub('[^a-z ]','',doc)
    tokens=doc.split()
    newdoc=""
    for token in tokens:
        if token not in sw:
            newdoc=newdoc+token+" "
    return newdoc.strip()
final_corpus=list(map(text_cleaning,corpus))
train_set=[]
for tup in zip(final_corpus,target):
    train_set.append(tup)
#train_set
model=NaiveBayesClassifier(train_set)

def predict_single(entry_user,lbl_result):
    sample=entry_user.get("1.0", "end-1c")
    final_sample=list(map(text_cleaning,[sample]))
    res=model.classify(final_sample)
    messagebox.showinfo('Prediction',res)

    
win=Tk()
win.state("zoomed")
win.configure(bg="powder blue")
win.resizable(width=False,height=False)

title=Label(win,text="News type classification",font=('arial',40,'bold','underline'),bg="powder blue")
title.pack()

date=Label(win,text=f"{datetime.now().date()}",font=('arial',15,'bold'),fg='black',bg="powder blue")
date.place(relx=.90,rely=.10)

def mainscreen():
    frm=Frame(win)
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    frm.configure(bg="powder blue")

    def new():
        frm.destroy()
        newscreen()
    def fp():
        frm.destroy()
        fpscreen()
    def login():
        acn=e_acn.get()
        pwd=e_pass.get()
        
    lbl_acn=Label(frm,text="username",font=('arial',20,'bold'),bg="powder blue")
    lbl_acn.place(relx=.3,rely=.2)
    
    e_user=Entry(frm,font=('arial',20,'bold'),bd=5)
    e_user.place(relx=.4,rely=.2)
    e_user.focus()
    
    lbl_pass=Label(frm,text="Password",font=('arial',20,'bold'),bg="powder blue")
    lbl_pass.place(relx=.3,rely=.3)
    
    e_pass=Entry(frm,font=('arial',20,'bold'),bd=5,show="*")
    e_pass.place(relx=.4,rely=.3)
    
    btn_login=Button(frm,command=lambda:newscreen(e_user,e_pass),text="login",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.45,rely=.5)

def newscreen(entry_user=None,entry_pass=None):
    if(entry_user!=None and entry_pass!=None):
        user=entry_user.get()
        pwd=entry_pass.get()
    else:
        user="admin"
        pwd="admin"
    if(len(user)==0 or len(pwd)==0):
        messagebox.showwarning("validation","Please fill both fields")
        return
    else:
        if(user=="admin" or pwd=="admin"):
            frm=Frame(win,bg='powder blue')
            frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
            
            
            btn_single=Button(frm,command=lambda:single_news_screen(),text="Single news Prediction",font=('',20,'bold'),bd=10,width=25)
            btn_single.place(relx=.33,rely=.2)


            btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
            btn_logout.place(relx=.9,rely=0)
        else:
            messagebox.showerror("Fail","Invalid Username/Password")

def single_news_screen():
    frm=Frame(win,bg='powder blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="Enter news:",font=('',20,'bold'),bg='green')
    lbl_user.place(relx=.26,rely=.3)

    entry_user=Text(frm,font=('',20),bd=10,width=25,height=4)
    entry_user.place(relx=.45,rely=.2)
    entry_user.focus()

    lbl_result=Label(frm,text="Prediction:",font=('',20,'bold'),bg='green')
    lbl_result.place(relx=.26,rely=.55)

    btn_login=Button(frm,command=lambda:predict_single(entry_user,lbl_result),text="predict",font=('',20,'bold'),bd=10,width=8)
    btn_login.place(relx=.47,rely=.46)

    btn_back=Button(frm,command=lambda:mainscreen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)


    
mainscreen()
win.mainloop()